# 🎯 Day 3 프로젝트: MLOps 파이프라인 구축

## 프로젝트 개요

지금까지 배운 내용을 종합하여 완전한 MLOps 파이프라인을 구축합니다.

### 요구사항

1. **데이터 파이프라인**: 데이터 로드 및 전처리
2. **실험 추적**: MLflow를 사용한 실험 관리
3. **모델 학습**: 하이퍼파라미터 튜닝 포함
4. **모델 평가**: 성능 기준에 따른 배포 결정
5. **모델 배포**: KServe를 사용한 자동 배포
6. **모니터링**: 배포된 모델 상태 확인

### 파이프라인 구조

```
┌─────────────┐   ┌─────────────┐   ┌─────────────────┐
│  load_data  │──▶│  preprocess │──▶│ hyperparameter  │
└─────────────┘   └─────────────┘   │    search       │
                                    └────────┬────────┘
                                             │
                                             ▼
                                    ┌─────────────────┐
                                    │   train_best    │
                                    └────────┬────────┘
                                             │
                                             ▼
                                    ┌─────────────────┐
                                    │    evaluate     │
                                    └────────┬────────┘
                                             │
                              ┌──────────────┴──────────────┐
                              ▼                             ▼
                       ┌─────────────┐               ┌─────────────┐
                       │   deploy    │               │    alert    │
                       └─────────────┘               └─────────────┘
```

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install kfp==1.8.22 mlflow==2.9.2 scikit-learn pandas numpy -q
print("✅ 패키지 설치 완료!")

In [ ]:
# Import
import kfp
from kfp import dsl
from kfp.components import create_component_from_func
from kfp import compiler
import os

print(f"KFP Version: {kfp.__version__}")

In [ ]:
# 환경 변수 설정 (본인의 설정으로 변경하세요!)
TEAM_NAME = "team-XX"  # TODO: 팀 이름으로 변경
USER_NAMESPACE = os.getenv('NAMESPACE', 'kubeflow-user01')  # TODO: 본인 네임스페이스로 변경

MLFLOW_TRACKING_URI = os.getenv(
    'MLFLOW_TRACKING_URI',
    'http://mlflow-server-service.mlflow-system.svc.cluster.local:5000'
)

print(f"Team: {TEAM_NAME}")
print(f"Namespace: {USER_NAMESPACE}")
print(f"MLflow URI: {MLFLOW_TRACKING_URI}")

---

## 2. 컴포넌트 구현

### 2.1 데이터 로드 컴포넌트

**TODO**: 데이터 로드 컴포넌트를 완성하세요.

In [ ]:
@create_component_from_func
def load_data(dataset_name: str = "california") -> str:
    """
    데이터를 로드합니다.
    
    TODO:
    1. sklearn에서 California Housing 데이터셋 로드
    2. DataFrame으로 변환
    3. CSV로 저장
    4. 저장 경로 반환
    """
    import pandas as pd
    # TODO: sklearn에서 데이터 로드
    # from sklearn.datasets import ...
    
    output_path = "/tmp/raw_data.csv"
    
    # TODO: 데이터 저장
    # df.to_csv(output_path, index=False)
    
    return output_path

### 2.2 전처리 컴포넌트

**TODO**: 전처리 컴포넌트를 완성하세요.

In [ ]:
@create_component_from_func
def preprocess(data_path: str, test_size: float = 0.2) -> str:
    """
    데이터 전처리를 수행합니다.
    
    TODO:
    1. CSV 로드
    2. Train/Test 분할
    3. StandardScaler로 정규화
    4. numpy 배열로 저장
    5. 저장 디렉토리 반환
    """
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    import os
    
    output_dir = "/tmp/processed"
    os.makedirs(output_dir, exist_ok=True)
    
    # TODO: 전처리 로직 구현
    
    return output_dir

### 2.3 하이퍼파라미터 탐색 컴포넌트 (선택)

**TODO**: 여러 하이퍼파라미터 조합을 테스트하는 컴포넌트를 구현하세요.

In [ ]:
@create_component_from_func
def hyperparameter_search(
    data_dir: str,
    mlflow_tracking_uri: str,
    experiment_name: str
) -> str:
    """
    하이퍼파라미터 탐색을 수행합니다.
    
    TODO:
    1. 여러 하이퍼파라미터 조합 정의
    2. 각 조합으로 모델 학습 및 평가
    3. MLflow에 모든 실험 기록
    4. 최적 하이퍼파라미터를 JSON으로 반환
    """
    import json
    
    # TODO: 하이퍼파라미터 탐색 구현
    # 힌트: n_estimators: [50, 100, 200], max_depth: [5, 10, 15]
    
    best_params = {
        "n_estimators": 100,
        "max_depth": 10
    }
    
    return json.dumps(best_params)

### 2.4 모델 학습 컴포넌트

**TODO**: MLflow 연동된 모델 학습 컴포넌트를 완성하세요.

In [ ]:
@create_component_from_func
def train_model(
    data_dir: str,
    best_params: str,
    mlflow_tracking_uri: str,
    experiment_name: str
) -> str:
    """
    최적 하이퍼파라미터로 모델을 학습합니다.
    
    TODO:
    1. 데이터 로드
    2. MLflow 설정
    3. 파라미터 로깅
    4. 모델 학습
    5. 메트릭 로깅 (MSE, RMSE, R2)
    6. 모델 저장
    7. Run ID 반환
    """
    import json
    import numpy as np
    import mlflow
    import mlflow.sklearn
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    import os
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    
    # TODO: 모델 학습 및 MLflow 로깅 구현
    
    run_id = "placeholder"
    return run_id

### 2.5 모델 평가 컴포넌트

**TODO**: 배포 결정을 내리는 평가 컴포넌트를 완성하세요.

In [ ]:
@create_component_from_func
def evaluate_model(
    run_id: str,
    mlflow_tracking_uri: str,
    r2_threshold: float = 0.8
) -> str:
    """
    모델 성능을 평가하고 배포 결정을 내립니다.
    
    TODO:
    1. MLflow에서 Run 정보 가져오기
    2. R2 메트릭 확인
    3. 임계값과 비교
    4. "deploy" 또는 "skip" 반환
    """
    import mlflow
    import os
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    
    # TODO: 평가 로직 구현
    
    decision = "deploy"  # or "skip"
    return decision

### 2.6 배포/알림 컴포넌트

In [ ]:
@create_component_from_func
def deploy_model(run_id: str, model_name: str, namespace: str, mlflow_tracking_uri: str):
    """
    모델을 KServe로 배포합니다.
    
    힌트: Kubernetes Python 클라이언트 사용
    """
    print(f"Deploying model: {model_name}")
    print(f"Namespace: {namespace}")
    print(f"Run ID: {run_id}")
    # TODO: KServe InferenceService 생성


@create_component_from_func
def send_alert(run_id: str, message: str = "Model did not meet threshold"):
    """성능 미달 알림을 보냅니다."""
    print(f"⚠️ ALERT: {message}")
    print(f"Run ID: {run_id}")

---

## 3. 파이프라인 정의

**TODO**: 위에서 정의한 컴포넌트들을 연결하여 파이프라인을 완성하세요.

In [ ]:
@dsl.pipeline(
    name=f'{TEAM_NAME}-mlops-pipeline',
    description='MLOps Project Pipeline'
)
def project_pipeline(
    dataset_name: str = "california",
    mlflow_tracking_uri: str = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000",
    experiment_name: str = "project-experiment",
    model_name: str = "project-model",
    namespace: str = "kubeflow-user01",
    r2_threshold: float = 0.8
):
    """
    TODO: 파이프라인 구현
    
    1. load_data 호출
    2. preprocess 호출 (load_data 출력 사용)
    3. hyperparameter_search 호출 (선택)
    4. train_model 호출
    5. evaluate_model 호출
    6. dsl.Condition으로 조건부 배포
    """
    
    # Step 1: 데이터 로드
    load_task = load_data(dataset_name=dataset_name)
    
    # Step 2: 전처리
    # TODO: preprocess 컴포넌트 연결
    
    # Step 3: 하이퍼파라미터 탐색 (선택)
    # TODO: hyperparameter_search 컴포넌트 연결
    
    # Step 4: 모델 학습
    # TODO: train_model 컴포넌트 연결
    
    # Step 5: 모델 평가
    # TODO: evaluate_model 컴포넌트 연결
    
    # Step 6: 조건부 배포
    # TODO: dsl.Condition 사용하여 조건부 분기 구현
    # with dsl.Condition(evaluate_task.output == "deploy"):
    #     deploy_model(...)
    # with dsl.Condition(evaluate_task.output == "skip"):
    #     send_alert(...)
    
    pass  # TODO: 위의 TODO들을 구현한 후 이 줄 삭제

print("✅ 파이프라인 정의 완료!")

---

## 4. 파이프라인 컴파일 및 실행

In [ ]:
# 컴파일
pipeline_file = f'{TEAM_NAME}_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=project_pipeline,
    package_path=pipeline_file
)

print(f"✅ Pipeline compiled: {pipeline_file}")

In [ ]:
# 실행
try:
    client = kfp.Client()
    print(f"✅ KFP Client connected")
    
    run = client.create_run_from_pipeline_func(
        project_pipeline,
        arguments={
            'dataset_name': 'california',
            'experiment_name': f'{TEAM_NAME}-experiment',
            'model_name': f'{TEAM_NAME}-model',
            'namespace': USER_NAMESPACE,
            'r2_threshold': 0.75
        },
        experiment_name=f'{TEAM_NAME}-project',
        run_name=f'{TEAM_NAME}-run-001'
    )
    
    print(f"\n✅ Pipeline submitted!")
    print(f"   Run ID: {run.run_id}")
    
except Exception as e:
    print(f"⚠️ Could not submit: {e}")
    print("\n💡 Upload YAML manually via Kubeflow UI")

---

## 5. 검증 체크리스트

발표 전 확인사항:

- [ ] 파이프라인이 오류 없이 컴파일되는가?
- [ ] 모든 컴포넌트가 정상 실행되는가?
- [ ] MLflow에 실험이 기록되는가?
- [ ] 조건부 분기가 올바르게 작동하는가?
- [ ] 모델 배포가 성공하는가?

---

## 📋 발표 준비

### 발표 내용 (15분)

1. **파이프라인 개요** (2분)
   - 전체 구조 설명
   - 각 컴포넌트 역할

2. **구현 하이라이트** (5분)
   - 핵심 코드 설명
   - 어려웠던 부분과 해결 방법

3. **데모** (5분)
   - 파이프라인 실행
   - MLflow UI 결과 확인
   - 배포된 모델 테스트

4. **Q&A** (3분)